In [1]:
import gdspy
import numpy as np
from scipy.spatial import ConvexHull

Load the gds file and create a target file to save edited design

In [2]:
gdsii = gdspy.GdsLibrary()
file = '/Users/wendy/Desktop/Wendy-qiskit-code/designs/export_test_0506_LL_test_200buff.gds'
#read the design file to meet compliance
gdsii.read_gds(file, units = 'import')

#create a new file 
gds_new = gdspy.GdsLibrary(unit = 1e-6, precision = 5e-10)

create a new cell to put the gds design

In [3]:
top = gds_new.new_cell('TOP')

Specify layer numbers

In [4]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60
chip_edge_layer = 703
text_layer1 = 98

layer_numbers  = [qubit_layer,junction_layer,ab_layer,ab_square_layer,junction_area_layer]

specify target datatype

In [5]:
data_type = 0
ground_plane_data_type = [0,10,11,101]

In [6]:
#snap to grid function that meets the layer requirements 
def snap_to_grid(coord, grid_size, limit = 10.15*1e4):
    new = (coord/grid_size).astype(int)
    res = []
    for (x,y) in new:
        if x > 0 and y>0:
            pass
        elif x > 0 and y<0:
            y -= 1
        elif x < 0 and y>0:
            x -= 1
        else:
            x -= 1
            y -= 1
        x = max(min(x, limit), -limit)
        y = max(min(y, limit), -limit)
        res += [[x * grid_size, y*grid_size]]

    return np.array(res)

In [7]:
#snap to grid function that meets the layer requirements 
def snap_to_grid1(coord, grid_size):
    new = np.round((coord/grid_size))*grid_size
    return new

In [8]:
import numpy as np

def remove_duplicate_points(polygon):
    """
    Remove duplicate points from the polygon represented as a NumPy array.
    """
    unique_rows = np.unique(polygon, axis=0)
    return unique_rows

def compute_area(polygon):
    """
    Compute the area of a polygon using Shoelace formula.
    """
    n = len(polygon)
    if n <= 2:
        return 0

    x = polygon[:, 0]
    y = polygon[:, 1]

    return 0.5 * abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def process_polygon(polygon):
    """
    Remove duplicate points from the given polygon and then return the polygon 
    if it has more than 2 points and non-zero area. Otherwise, return None.
    """
    deduplicated_polygon = remove_duplicate_points(polygon)
    
    if len(deduplicated_polygon) > 2 and compute_area(deduplicated_polygon) != 0:
        return deduplicated_polygon
    else:
        return None

# Example usage:
polygon = np.array([[0, 0], [1, 1], [1, 0], [0, 0]])
result = process_polygon(polygon)
print(result)


[[0 0]
 [1 0]
 [1 1]]


In [9]:
remove_duplicate_points(polygon)

array([[0, 0],
       [1, 0],
       [1, 1]])

In [10]:
qubit_layer_grid = 50*1e-6
junction_layer_grid = 5*1e-6

Get the polygons from the old design

In [11]:
polys = gdsii.top_level()[0].get_polygons(by_spec = True)

In [12]:
keys = list(polys.keys())

In [13]:
keys

[(5, 10), (5, 11), (5, 100), (20, 10), (60, 10), (30, 10), (31, 10)]

Extract all the data types for a given layer number

In [14]:
datatypes = {}
for i in keys:
    try:
        print(datatypes[i[0]])
    except:
        datatypes[i[0]] = [i[1]]
    else:
        datatypes[i[0]].append(i[1])


[10]
[10, 11]


In [15]:
datatypes[5] = [10,11,100]

enumerate over layer numbers and save the polygon into the new design with desired layer number

In [16]:
import numpy as np
from shapely.geometry import Polygon

def compute_area(coordinates):
    """Compute the area of a polygon given its coordinates."""
    x = coordinates[:, 0]
    y = coordinates[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def order_polygon_coordinates_using_shapely(coordinates):
    # Create a shapely polygon

    polygon = Polygon(coordinates)
    
    # Access the exterior coordinates of the polygon
    exterior_coords = np.array(polygon.exterior.coords[:-1])  # omitting the last point since it's same as the first

    # Check for zero area
    if np.isclose(compute_area(exterior_coords), 0.0, atol=1e-10):
        return None

    return exterior_coords

# Example list of coordinates
coordinates = np.array([
    [0, 0],
    [3, 1],
    [1, 3],
    [1, 1],
    [1, 1]  # a duplicate point
])

ordered_coords = order_polygon_coordinates_using_shapely(coordinates)
if ordered_coords is not None:
    print(ordered_coords)
else:
    print("The polygon has zero area.")


[[0. 0.]
 [3. 1.]
 [1. 3.]
 [1. 1.]
 [1. 1.]]


In [17]:
def remove_adjacent_duplicates(coordinates):
    i = 0
    # Initialize the result with the first coordinate
    result = [coordinates[0]]
    # Loop over the coordinates
    for coord in coordinates[1:]:
        # If the current coordinate is not the same as the last one in the result, add it to the result
        if not np.array_equal(coord, result[-1]):
            result.append(coord)
        else:
            i+=1
            print("Removed a point ",i)
    return np.array(result)

# Example list of coordinates
coordinates = np.array([
    [0, 0],
    [3, 1],
    [1, 3],
    [1, 1],
    [1, 1]  # a duplicate point
])

coordinates = remove_adjacent_duplicates(coordinates)
print(coordinates)

Removed a point  1
[[0 0]
 [3 1]
 [1 3]
 [1 1]]


In [18]:
import numpy as np

def compute_area(coordinates):
    """Compute the area of a polygon given its coordinates."""
    x = coordinates[:, 0]
    y = coordinates[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def order_polygon_coordinates(coordinates):
    # Remove duplicate points
    unique_coordinates = np.array(list(set(map(tuple, coordinates))))
    
    if len(unique_coordinates) <= 2:  # A polygon with 2 or fewer unique points will have zero area
        return None

    # Calculate centroid
    centroid = unique_coordinates.mean(axis=0)
    
    # Sorting by angle using centroid
    sorted_coordinates = sorted(unique_coordinates, 
                               key=lambda p: np.arctan2(p[1] - centroid[1], p[0] - centroid[0]))

    ordered_array = np.array(sorted_coordinates)

    # Check for zero area
    if np.isclose(compute_area(ordered_array), 0.0, atol=1e-10):
        return None

    return ordered_array

# Example list of coordinates
coordinates = np.array([
    [0, 0],
    [3, 1],
    [1, 3],
    [1, 1]
])

ordered_coords = order_polygon_coordinates(coordinates)
if ordered_coords is not None:
    print(ordered_coords)
else:
    print("The polygon has zero area.")


[[1 1]
 [0 0]
 [3 1]
 [1 3]]


In [19]:
coordinates

array([[0, 0],
       [3, 1],
       [1, 3],
       [1, 1]])

In [20]:
remove_adjacent_duplicates((polygon))

array([[0, 0],
       [1, 1],
       [1, 0],
       [0, 0]])

In [21]:
datatypes[5] = []

In [22]:
t = 0
for i, layer in enumerate(layer_numbers):
    for j in datatypes[layer]:
        source_polyons = polys.get((layer,j),[])

        for polygon in source_polyons:
            if (layer ==5) or (layer == 31) or (layer == 30):
                polygon = snap_to_grid(polygon, qubit_layer_grid)
            elif (layer == 20) or (layer == 60):
                polygon = snap_to_grid(polygon, junction_layer_grid)
            polygon = order_polygon_coordinates_using_shapely(polygon)
            
            if polygon is None:
                t+=1
                print('This is the {}th polygon with zero area'.format(t))
                continue
            else:
                p =gdspy.Polygon(polygon*(1e-3/1e-6), layer = int(layer), datatype = 0)
                
            # print(cell_names[i])
            top.add(p)

This is the 1th polygon with zero area
This is the 2th polygon with zero area
This is the 3th polygon with zero area
This is the 4th polygon with zero area
This is the 5th polygon with zero area
This is the 6th polygon with zero area
This is the 7th polygon with zero area
This is the 8th polygon with zero area
This is the 9th polygon with zero area
This is the 10th polygon with zero area
This is the 11th polygon with zero area
This is the 12th polygon with zero area
This is the 13th polygon with zero area
This is the 14th polygon with zero area
This is the 15th polygon with zero area
This is the 16th polygon with zero area
This is the 17th polygon with zero area
This is the 18th polygon with zero area
This is the 19th polygon with zero area
This is the 20th polygon with zero area
This is the 21th polygon with zero area
This is the 22th polygon with zero area
This is the 23th polygon with zero area
This is the 24th polygon with zero area
This is the 25th polygon with zero area
This is t

This is the 154th polygon with zero area
This is the 155th polygon with zero area
This is the 156th polygon with zero area
This is the 157th polygon with zero area
This is the 158th polygon with zero area
This is the 159th polygon with zero area
This is the 160th polygon with zero area
This is the 161th polygon with zero area
This is the 162th polygon with zero area
This is the 163th polygon with zero area
This is the 164th polygon with zero area
This is the 165th polygon with zero area
This is the 166th polygon with zero area
This is the 167th polygon with zero area
This is the 168th polygon with zero area
This is the 169th polygon with zero area
This is the 170th polygon with zero area
This is the 171th polygon with zero area
This is the 172th polygon with zero area
This is the 173th polygon with zero area
This is the 174th polygon with zero area
This is the 175th polygon with zero area
This is the 176th polygon with zero area
This is the 177th polygon with zero area
This is the 178t

In [23]:
temp_layer = 5

In [24]:
t = 0
for i, layer in enumerate([5]):
    for j in [10,11,100]:
        source_polyons = polys.get((layer,j),[])

        for polygon in source_polyons:
            if (layer ==5) or (layer == 31) or (layer == 30):
                polygon = snap_to_grid(polygon, qubit_layer_grid)
            elif (layer == 20) or (layer == 60):
                polygon = snap_to_grid(polygon, junction_layer_grid)
            polygon = order_polygon_coordinates_using_shapely(polygon)
            
            if polygon is None:
                t+=1
                print('This is the {}th polygon with zero area'.format(t))
                continue
            else:
                p =gdspy.Polygon(polygon*(1e-3/1e-6), layer = int(temp_layer), datatype = 0)
                
            # print(cell_names[i])
            top.add(p)

This is the 1th polygon with zero area
This is the 2th polygon with zero area
This is the 3th polygon with zero area


In [25]:
layer = 5
j = 101
source_polyons = polys.get((layer,j),[])
for polygon in source_polyons:
    if (layer ==5) or (layer == 31) or (layer == 30):
        polygon = snap_to_grid(polygon, qubit_layer_grid)
    elif (layer == 20) or (layer == 60):
        polygon = snap_to_grid(polygon, junction_layer_grid)
    polygon = order_polygon_coordinates_using_shapely(polygon)
    
    if polygon is None:
        t+=1
        print('This is the {}th polygon with zero area'.format(t))
        continue
    else:
        p =gdspy.Polygon(polygon*(1e-3/1e-6), layer = int(100), datatype = 0)
        
    # print(cell_names[i])
    top.add(p)

add the chip edge

In [26]:
rectangle = gdspy.Polygon(1e3*np.array([[-10.15/2,10.15/2],[10.15/2,10.15/2],[10.15/2,-10.15/2],[-10.15/2,-10.15/2]]), layer = int(chip_edge_layer))
top.add(rectangle)

Write necessary information in the information cells

In [27]:
htest1 = gdspy.Text('Email',1/2*1000,(-5000,4000), layer = text_layer1)
htest2 = gdspy.Text('wendywan@stanford.edu',1/2*1000, (-5000,2000), layer = text_layer1)
htest3 = gdspy.Text('Jc = 0.1uA/um2',1/2*1000,(-5000,0), layer = text_layer1)

In [28]:
top.add(htest1)
top.add(htest2)
top.add(htest3)

In [29]:
cell = gds_new.cells['TOP']

In [30]:
layer_to_delete = 100  # Replace with your layer number

# Remove elements from the specified layer
# This creates a list of polygons not on the layer to delete and then replaces the original polygons
top.polygons = [poly for poly in cell.polygons if poly.layers[0] != layer_to_delete]
layer_to_delete = 101  # Replace with your layer number

# Remove elements from the specified layer
# This creates a list of polygons not on the layer to delete and then replaces the original polygons
top.polygons = [poly for poly in cell.polygons if poly.layers[0] != layer_to_delete]

In [31]:
gdsii = gdspy.GdsLibrary(unit = 1e-6, precision = 5e-10)
gdsii.add(cell)

In [32]:
gdsii.write_gds('design_final_0116_15_2.gds')

In [33]:
raise SystemExit("Stop right there!")

SystemExit: Stop right there!

/Users/wendy/miniconda3/envs/qiskit/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
gdsii = gdspy.GdsLibrary()
gdsii.read_gds('design_final_0116.gds', units = 'import')

# Extract the layers. Replace 'cell_name' with the name of your cell
cell = gdsii.cell_dict['cell_name']
layer1 = cell.get_polygons(by_spec=True)[(1, 0)]  # Layer 1 (assuming datatype 0)
layer2 = cell.get_polygons(by_spec=True)[(2, 0)]  # Layer 2 (assuming datatype 0)

# Perform the boolean subtraction: subtract layer1 from layer2
result = gdspy.boolean(layer2, layer1, 'not', layer=3, datatype=0)  # You can choose your own layer and datatype for the result

# Add the result to the cell
cell.add(result)

# Save the result to a new GDS file
gdsii.write_gds('result.gds')

In [ ]:
np.round(-1.1)

-1.0

In [ ]:
polygon

array([[ 4.171   , -4.529545],
       [ 4.171   , -4.531605],
       [ 4.1695  , -4.531605],
       [ 4.169   , -4.533075],
       [ 4.169   , -4.528075],
       [ 4.1695  , -4.529545]])

array([[ 4.171   , -4.529545],
       [ 4.171   , -4.5316  ],
       [ 4.1695  , -4.5316  ],
       [ 4.169   , -4.53307 ],
       [ 4.169   , -4.52807 ],
       [ 4.1695  , -4.529545]])